# Julia言語で入門するプログラミング

[Julia言語で入門するプログラミング](https://muuumin.net/julia-intro-prog/)

## リファクタリング

### 構造体
現在、戦闘に関連するパラメーター（変数）は以下の6つを使っている

```julia
モンスターHP = 30
モンスター攻撃力 = 10
モンスター防御力 = 10
勇者HP = 30
勇者攻撃力 = 10
勇者防御力 = 10
```

これは本来、モンスター／プレイヤーの2パターンと、それぞれが持つHP／攻撃力／防御力の3パターンである

こういった場合 **構造体** を使って、複数の変数を1つにまとめるとすっきり記述できるようになる

In [1]:
struct キャラクター
    HP::Int
    攻撃力::Int
    防御力::Int
end

# HP: 30, 攻撃力: 20, 防御力: 10 のキャラクターとして勇者を定義
勇者 = キャラクター(30, 20, 10)

@show 勇者
println("勇者HP: $(勇者.HP)")
println("勇者攻撃力: $(勇者.攻撃力)")
println("勇者防御力: $(勇者.防御力)")

勇者 = キャラクター(30, 20, 10)
勇者HP: 30
勇者攻撃力: 20
勇者防御力: 10


### 可変構造体と module 宣言
構造体はデフォルトで immutable である

すなわち、構造体内部の変数を後から変更することはできない

ゲームにおいては、HPなどの変数の値はどんどん変更されていくため、これでは困る

こういった場合は `mutable struct` で可変構造体を宣言する

この可変構造体を使って、前回の `main` 関数をリファクタリングしていくが、ここで一つ問題が発生する

構造体のような型宣言は、Julia の REPL 上で同名の型を再定義できないのである

そのため、今のまま `mutable struct キャラクター ... end` のように構造体を宣言しようとすると、`invalid redefinition of constant キャラクター` というようなエラーが発生する

In [2]:
mutable struct キャラクター
    HP::Int
    攻撃力::Int
    防御力::Int
end

LoadError: invalid redefinition of constant キャラクター

このような問題を回避するテクニックとして `module` を使う方法がある

`module` は本来、特定の機能に関連する型宣言・関数・変数等を一つのパッケージにまとめるための機能だが、`module` 自体は REPL 上で再定義可能なため、型宣言などを再定義するのに使うことができる

今回は、このゲームプログラムを `RPG` module として定義することにして、前回の `main` 関数をリファクタリングしていく

In [3]:
# このゲームプログラムを RPG module としてまとめる
module RPG

using Test

"HP計算関数"
function HP計算(HP::Int, ダメージ量::Int)
    ダメージ後のHP = HP - ダメージ量
    if ダメージ後のHP < 0
        return 0
    end
    return ダメージ後のHP
end

"ダメージ計算関数"
function ダメージ計算(攻撃力::Int, 防御力::Int)
    if 攻撃力 < 0 || 防御力 < 0
        throw(DomainError((攻撃力, 防御力), "攻撃力, 防御力は正の値である必要があります")) 
    end
    if 防御力 === 0
        throw(DomainError(防御力, "防御力は0より大きい値である必要があります")) 
    end
    ダメージ量 = round(Int, 10 * 攻撃力 / 防御力)
end

"キャラクター可変構造体"
mutable struct キャラクター
    HP::Int
    攻撃力::Int
    防御力::Int
end

"""
    main(乱数生成関数::Function) ->  戦闘ターン数::Int,
                                    戦闘後の勇者HP::Int,
                                    戦闘後のモンスターHP::Int

自動テスト可能な main 関数
"""
function main(乱数生成関数::Function)
    # 各キャラクターのパラメータを構造体にまとめる
    モンスター = キャラクター(30, 10, 10)
    勇者 = キャラクター(30, 10, 10)

    戦闘ターン数 = 1

    println("モンスターに遭遇した！")
    println("戦闘開始！")

    while 勇者.HP > 0 && モンスター.HP > 0
        if 乱数生成関数() < 0.5
            # 50％の確率で勇者が先制攻撃
            println("----------")
            println("勇者の攻撃！")
            モンスターダメージ = ダメージ計算(勇者.攻撃力, モンスター.防御力)
            モンスター.HP = HP計算(モンスター.HP, モンスターダメージ)
            println("モンスターは $(モンスターダメージ) のダメージを受けた！")
            println("モンスターの残りHP：$(モンスター.HP)")

            if モンスター.HP === 0
                break
            end

            println("----------")
            println("モンスターの攻撃！")
            勇者ダメージ = ダメージ計算(モンスター.攻撃力, 勇者.防御力)
            勇者.HP = HP計算(勇者.HP, 勇者ダメージ)
            println("勇者は $(勇者ダメージ) のダメージを受けた！")
            println("勇者の残りHP：$(勇者.HP)")

            if 勇者.HP === 0
                break
            end
        else
            # 50％の確率でモンスターが先制攻撃
            println("----------")
            println("モンスターの攻撃！")
            勇者ダメージ = ダメージ計算(モンスター.攻撃力, 勇者.防御力)
            勇者.HP = HP計算(勇者.HP, 勇者ダメージ)
            println("勇者は $(勇者ダメージ) のダメージを受けた！")
            println("勇者の残りHP：$(勇者.HP)")

            if 勇者.HP === 0
                break
            end

            println("----------")
            println("勇者の攻撃！")
            モンスターダメージ = ダメージ計算(勇者.攻撃力, モンスター.防御力)
            モンスター.HP = HP計算(モンスター.HP, モンスターダメージ)
            println("モンスターは $(モンスターダメージ) のダメージを受けた！")
            println("モンスターの残りHP：$(モンスター.HP)")

            if モンスター.HP === 0
                break
            end
        end

        # 戦闘ターン数 を 1 増やす
        戦闘ターン数 += 1
    end

    # モンスターHPが 0 になっているかどうかで戦闘終了メッセージを変える
    if モンスター.HP === 0
        println("戦闘に勝利した！")
    else
        println("戦闘に敗北した・・・")
    end

    # テストしたい値を返す
    戦闘ターン数, 勇者.HP, モンスター.HP
end

#=
自動テスト: 乱数生成関数が出力する数列をもとに、戦闘ターン数と戦闘後の勇者・モンスターのHPをテストする

- [0.5, 0.4, 0.6]:
    1. モンスター(HP: 30) -> 勇者(HP: 30) 攻撃 ==> 勇者(HP: 20) -> モンスター(HP: 30) 攻撃
    2. 勇者(HP: 20) -> モンスター(HP: 20) 攻撃 ==> モンスター(HP: 10) -> 勇者(HP: 20) 攻撃
    3. モンスター(HP: 10) -> 勇者(HP: 10) 攻撃 ==> 戦闘終了: 勇者(HP: 0), モンスター(HP: 10)
- [0.4, 0.7, 0.3]:
    1. 勇者(HP: 30) -> モンスター(HP: 30) 攻撃 ==> モンスター(HP: 20) -> 勇者(HP: 30) 攻撃
    2. モンスター(HP: 20) -> 勇者(HP: 20) 攻撃 ==> 勇者(HP: 10) -> モンスター(HP: 20) 攻撃
    3. 勇者(HP: 10) -> モンスター(HP: 10) ==> 戦闘終了: 勇者(HP: 10), モンスター(HP: 0)
=#
function 疑似乱数生成高階関数(疑似乱数配列::Vector{<:Number})
    function 疑似乱数生成関数()
        popfirst!(疑似乱数配列)
    end
end

@testset "main関数" begin
    @test main(疑似乱数生成高階関数([0.5, 0.4, 0.6])) === (3, 0, 10)
    @test main(疑似乱数生成高階関数([0.4, 0.7, 0.3])) === (3, 10, 0)
end

end # module RPG

モンスターに遭遇した！
戦闘開始！
----------
モンスターの攻撃！
勇者は 10 のダメージを受けた！
勇者の残りHP：20
----------
勇者の攻撃！
モンスターは 10 のダメージを受けた！
モンスターの残りHP：20
----------
勇者の攻撃！
モンスターは 10 のダメージを受けた！
モンスターの残りHP：10
----------
モンスターの攻撃！
勇者は 10 のダメージを受けた！
勇者の残りHP：10
----------
モンスターの攻撃！
勇者は 10 のダメージを受けた！
勇者の残りHP：0
戦闘に敗北した・・・
モンスターに遭遇した！
戦闘開始！
----------
勇者の攻撃！
モンスターは 10 のダメージを受けた！
モンスターの残りHP：20
----------
モンスターの攻撃！
勇者は 10 のダメージを受けた！
勇者の残りHP：20
----------
モンスターの攻撃！
勇者は 10 のダメージを受けた！
勇者の残りHP：10
----------
勇者の攻撃！
モンスターは 10 のダメージを受けた！
モンスターの残りHP：10
----------
勇者の攻撃！
モンスターは 10 のダメージを受けた！
モンスターの残りHP：0
戦闘に勝利した！
Test Summary: | Pass  Total
main関数      |    2      2


Main.RPG

コードが多少すっきりしてきたが、まだ冗長な感じがする

これは、攻撃を行っている部分のコードがほぼコピペで出来ているためと思われる

実際のところ、攻撃しているのが勇者かモンスターかの違いだけで、その中の処理は全く同じである

そのため、この攻撃部分の処理を以下のような関数にまとめると、かなりすっきり書けそうである

```julia
function 攻撃処理!(攻撃者::キャラクター, 防御者::キャラクター)
    ...
end
```

Julia の関数名の慣例として、引数に渡した構造体の値を変更するような関数は `!` ("bang") をつけると良い

この "bang" マークがあることにより、関数の実行前後で、構造体内部変数に変更が生じていることが明確になる

今回の場合は、`攻撃処理!` 関数実行後、`防御者.HP` が変更されている（減っている）はずである

In [4]:
# HP計算, ダメージ計算 関数は使いまわしたいため退避しておく
import .RPG
HP計算 = RPG.HP計算
ダメージ計算 = RPG.ダメージ計算

# --------------------------------------------------
# RPG module の再定義
module RPG

using Test

# 親 module (RPG module の外側) に退避していた HP計算, ダメージ計算 関数を import
import ..HP計算, ..ダメージ計算

"キャラクター可変構造体"
mutable struct キャラクター
    名前::String # 追加
    HP::Int
    攻撃力::Int
    防御力::Int
end

"攻撃処理関数"
function 攻撃処理!(攻撃者::キャラクター, 防御者::キャラクター)
    println("----------")
    println("$(攻撃者.名前)の攻撃！")
    防御者ダメージ = ダメージ計算(攻撃者.攻撃力, 防御者.防御力)
    防御者.HP = HP計算(防御者.HP, 防御者ダメージ)
    println("$(防御者.名前)は $(防御者ダメージ) のダメージを受けた！")
    println("$(防御者.名前)の残りHP：$(防御者.HP)")
end

"main関数"
function main(乱数生成関数::Function)
    モンスター = キャラクター("モンスター", 30, 10, 10)
    勇者 = キャラクター("勇者", 30, 10, 10)

    戦闘ターン数 = 1

    println("モンスターに遭遇した！")
    println("戦闘開始！")

    while 勇者.HP > 0 && モンスター.HP > 0
        if 乱数生成関数() < 0.5
            # 50％の確率で勇者が先制攻撃
            攻撃処理!(勇者, モンスター)
            if モンスター.HP === 0
                break
            end

            攻撃処理!(モンスター, 勇者)
            if 勇者.HP === 0
                break
            end
        else
            # 50％の確率でモンスターが先制攻撃
            攻撃処理!(モンスター, 勇者)
            if 勇者.HP === 0
                break
            end

            攻撃処理!(勇者, モンスター)
            if モンスター.HP === 0
                break
            end
        end

        戦闘ターン数 += 1
    end

    if モンスター.HP === 0
        println("戦闘に勝利した！")
    else
        println("戦闘に敗北した・・・")
    end

    戦闘ターン数, 勇者.HP, モンスター.HP
end

# test
function 疑似乱数生成高階関数(疑似乱数配列::Vector{<:Number})
    function 疑似乱数生成関数()
        popfirst!(疑似乱数配列)
    end
end

@testset "main関数" begin
    @test main(疑似乱数生成高階関数([0.5, 0.4, 0.6])) === (3, 0, 10)
    @test main(疑似乱数生成高階関数([0.4, 0.7, 0.3])) === (3, 10, 0)
end

end # module RPG

モンスターに遭遇した！
戦闘開始！
----------
モンスターの攻撃！
勇者は 10 のダメージを受けた！
勇者の残りHP：20
----------
勇者の攻撃！
モンスターは 10 のダメージを受けた！
モンスターの残りHP：20
----------
勇者の攻撃！
モンスターは 10 のダメージを受けた！
モンスターの残りHP：10
----------
モンスターの攻撃！
勇者は 10 のダメージを受けた！
勇者の残りHP：10
----------
モンスターの攻撃！
勇者は 10 のダメージを受けた！
勇者の残りHP：0
戦闘に敗北した・・・
モンスターに遭遇した！
戦闘開始！
----------
勇者の攻撃！
モンスターは 10 のダメージを受けた！
モンスターの残りHP：20
----------
モンスターの攻撃！
勇者は 10 のダメージを受けた！
勇者の残りHP：20
----------
モンスターの攻撃！
勇者は 10 のダメージを受けた！
勇者の残りHP：10
----------
勇者の攻撃！
モンスターは 10 のダメージを受けた！
モンスターの残りHP：10
----------
勇者の攻撃！
モンスターは 10 のダメージを受けた！
モンスターの残りHP：0
戦闘に勝利した！
Test Summary: | Pass  Total
main関数      |    2      2


Main.RPG

だいぶすっきりしたコードになってきた

とはいえ、`while` ループの中がまだ少し長い

ここではもう少し踏み込んで、「行動順の決定」と「ゲームループ」部分を別関数に分けてみる

In [5]:
import .RPG
HP計算 = RPG.HP計算
ダメージ計算 = RPG.ダメージ計算
攻撃処理! = RPG.攻撃処理!
疑似乱数生成高階関数 = RPG.疑似乱数生成高階関数

# キャラクター構造体を一時退避
## Main.キャラクター は struct 型で定義済のため別の変数名で退避する必要がある
キャラクター構造体 = RPG.キャラクター

# --------------------------------------------------
module RPG

using Test

import ..HP計算, ..ダメージ計算, ..攻撃処理!, ..疑似乱数生成高階関数, ..キャラクター構造体

キャラクター = キャラクター構造体

"""
    行動順決定(勇者::キャラクター, モンスター::キャラクター, 乱数::Number)
        -> [(攻撃者::キャラクター, 防御者::キャラクター), ...]

行動順決定関数
"""
function 行動順決定(勇者::キャラクター, モンスター::キャラクター, 乱数::Number)
    if 乱数 < 0.5
        return [(勇者, モンスター), (モンスター, 勇者)]
    else
        return [(モンスター, 勇者), (勇者, モンスター)]
    end
end

"""
    ゲームループ!(勇者::キャラクター, モンスター::キャラクター, 乱数生成関数::Function)
        -> 戦闘ターン数::Int

ゲームループ処理関数
"""
function ゲームループ!(勇者::キャラクター, モンスター::キャラクター, 乱数生成関数::Function)
    戦闘ターン数 = 1

    while 勇者.HP > 0 && モンスター.HP > 0
        for (攻撃者, 防御者) in 行動順決定(勇者, モンスター, 乱数生成関数())
            攻撃処理!(攻撃者, 防御者)
            if 防御者.HP === 0
                return 戦闘ターン数 # ゲームループ終了
            end
        end
        戦闘ターン数 += 1
    end

    # 戦闘ターン数を返す
    戦闘ターン数
end

"main関数"
function main(乱数生成関数::Function)
    モンスター = キャラクター("モンスター", 30, 10, 10)
    勇者 = キャラクター("勇者", 30, 10, 10)

    println("モンスターに遭遇した！")
    println("戦闘開始！")

    戦闘ターン数 = ゲームループ!(勇者, モンスター, 乱数生成関数)

    if モンスター.HP === 0
        println("戦闘に勝利した！")
    else
        println("戦闘に敗北した・・・")
    end

    戦闘ターン数, 勇者.HP, モンスター.HP
end

# test
@testset "main関数" begin
    @test main(疑似乱数生成高階関数([0.5, 0.4, 0.6])) === (3, 0, 10)
    @test main(疑似乱数生成高階関数([0.4, 0.7, 0.3])) === (3, 10, 0)
end

end # module RPG

モンスターに遭遇した！
戦闘開始！
----------
モンスターの攻撃！
勇者は 10 のダメージを受けた！
勇者の残りHP：20
----------
勇者の攻撃！
モンスターは 10 のダメージを受けた！
モンスターの残りHP：20
----------
勇者の攻撃！
モンスターは 10 のダメージを受けた！
モンスターの残りHP：10
----------
モンスターの攻撃！
勇者は 10 のダメージを受けた！
勇者の残りHP：10
----------
モンスターの攻撃！
勇者は 10 のダメージを受けた！
勇者の残りHP：0
戦闘に敗北した・・・
モンスターに遭遇した！
戦闘開始！
----------
勇者の攻撃！
モンスターは 10 のダメージを受けた！
モンスターの残りHP：20
----------
モンスターの攻撃！
勇者は 10 のダメージを受けた！
勇者の残りHP：20
----------
モンスターの攻撃！
勇者は 10 のダメージを受けた！
勇者の残りHP：10
----------
勇者の攻撃！
モンスターは 10 のダメージを受けた！
モンスターの残りHP：10
----------
勇者の攻撃！
モンスターは 10 のダメージを受けた！
モンスターの残りHP：0
戦闘に勝利した！
Test Summary: | Pass  Total
main関数      |    2      2


Main.RPG

これで `main` 関数はだいぶすっきりしたのではないだろうか

このように、一つまとまりになっているデータや処理は、構造体や関数に上手く分離していくときれいなコードを書けるようになる

ゲームとしての完成度はともかく、これで一通りのリファクタリングは完了である